In [5]:
import requests
import pandas as pd
import time
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv('TMDB_API_KEY')

directors = ['Justine Triet', 'Martin Scorsese', 'Christopher Nolan', 'Yorgos Lanthimos', 'Jonathan Glazer', "Daniel Kwan", "Daniel Scheinert", "Steven Spielberg", "Todd Field", "Martin McDonagh", "Ruben Östlund", "Jane Campion", "Paul Thomas Anderson", "Ryusuke Hamaguchi", "Kenneth Branagh", "Chloé Zhao", "David Fincher", "Emerald Fennell", "Thomas Vinterberg", "Lee Isaac Chung", "Bong Joon Ho","Quentin Tarantino", "Martin Scorsese", "Sam Mendes", "Todd Phillips", "Alfonso Cuarón", "Yorgos Lanthimos", "Spike Lee", "Paweł Pawlikowski", "Adam McKay", "Guillermo del Toro", "Jordan Peele", "Christopher Nolan", "Greta Gerwig", "Paul Thomas Anderson", "Damien Chazelle", "Barry Jenkins", "Mel Gibson", "Denis Villeneuve", "Kenneth Lonergan", "Tom McCarthy", "Adam McKay", "George Miller", "Lenny Abrahamson", "Richard Linklater", "Bennett Miller", "Wes Anderson", "Morten Tyldum", "Alfonso Cuarón", "David O. Russell", "Steve McQueen", "Martin Scorsese", "Alexander Payne", "Ang Lee", "Steven Spielberg", "David O. Russell", "Benh Zeitlin", "Michael Haneke", "Michel Hazanavicius", "Martin Scorsese", "Woody Allen", "Terrence Malick", "Alexander Payne", "Tom Hooper", "David Fincher", "Ethan Coen", "Joel Coen", "Darren Aronofsky", "David O. Russell", "Kathryn Bigelow", "James Cameron", "Quentin Tarantino", "Jason Reitman", "Lee Daniels", "Danny Boyle", "Gus Van Sant", "David Fincher", "Ron Howard", "Stephen Daldry", "Ethan Coen", "Joel Coen", "Jason Reitman", "Tony Gilroy", "Julian Schnabel", "Paul Thomas Anderson", "Martin Scorsese", "Clint Eastwood", "Stephen Frears", "Paul Greengrass", "Alejandro González Iñárritu", "Ang Lee", "Steven Spielberg", "George Clooney", "Paul Haggis", "Bennett Miller", "Clint Eastwood", "Martin Scorsese", "Mike Leigh", "Alexander Payne", "Taylor Hackford", "Peter Jackson", "Sofia Coppola", "Clint Eastwood", "Peter Weir", "Fernando Meirelles", "Roman Polanski", "Martin Scorsese", "Rob Marshall", "Pedro Almodóvar", "Stephen Daldry", "Ron Howard", "Ridley Scott", "Robert Altman", "David Lynch", "Ang Lee", "Steven Soderbergh", "Ridley Scott", "Steven Soderbergh", "Ang Lee", "Stephen Daldry", "Sam Mendes", "M. Night Shyamalan", "Spike Jonze", "Michael Mann", "Lasse Hallström", "Steven Spielberg", "John Madden", "Roberto Benigni", "Terrence Malick", "Peter Weir", "James Cameron", "Gus Van Sant", "Curtis Hanson", "Peter Cattaneo", "Atom Egoyan", "Anthony Minghella", "Joel Coen", "Mike Leigh", "Scott Hicks", "Milos Forman", "Mel Gibson", "Tim Robbins", "Mike Figgis", "Chris Noonan", "Michael Radford", "Robert Zemeckis", "Quentin Tarantino", "Robert Redford", "Woody Allen", "Krzysztof Kieslowski", "Steven Spielberg", "Robert Altman", "James Ivory", "Jim Sheridan", "Jonathan Demme", "Oliver Stone", "John Singleton", "Ridley Scott", "Barry Levinson", "Kevin Costner", "Martin Scorsese", "Francis Ford Coppola", "Stephen Frears", "Barbet Schroeder", "Oliver Stone", "Woody Allen", "Kenneth Branagh", "Jim Sheridan", "Peter Weir", "Barry Levinson", "Bernardo Bertolucci", "Norman Jewison", "Adrian Lyne", "Lasse Hallström", "John Boorman", "Oliver Stone", "David Lynch", "Woody Allen", "Roland Joffé", "James Ivory", "Sydney Pollack", "John Huston", "Akira Kurosawa", "Hector Babenco", "Milos Forman", "David Lean", "William Wyler", "Sidney Lumet", "Robert Aldrich", "Fred Zinnemann", "Cecil B. DeMille", "John Ford", "William Wyler", "George Stevens", "Elia Kazan", "Joseph L. Mankiewicz", "William Wyler", "Frank Capra", "Michael Curtiz", "Norman Taurog", "Leo McCarey", "W.S. Van Dyke", "Frank Capra", "George Cukor", "Frank Borzage", "King Vidor", "Josef von Sternberg", "Norman Taurog", "Ernst Lubitsch", "Frank Lloyd", "Harry Beaumont", "Irving Cummings"]

unique_directors = list(set(directors))
unique_directors.sort()

In [6]:
all_movies = []

for director in unique_directors:
    response = requests.get(f"https://api.themoviedb.org/3/search/person?api_key={api_key}&query={director}")
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            director_id = data['results'][0]['id']

            url = f"https://api.themoviedb.org/3/person/{director_id}/movie_credits?api_key={api_key}"
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                directed_movies = [movie for movie in data['crew'] if movie['job'] == 'Director']

                for movie in directed_movies:
                    movie_id = movie['id']
                    movie_title = movie['title']
                    movie_release_date = movie['release_date']
                    # Obtém detalhes adicionais do filme
                    movie_details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}"
                    details_response = requests.get(movie_details_url)
                    if details_response.status_code == 200:
                        details_data = details_response.json()
                        duration = details_data['runtime']
                        genres = [genre['name'] for genre in details_data['genres']]
                        genre_names = ', '.join(genres)
                        release_year = details_data['release_date'][:4]

                        # Filtra para incluir apenas filmes com duração maior que 50 minutos
                        if duration and duration > 50:
                            all_movies.append({
                                'Movie Title': movie_title,
                                'Director': director,
                                'Genre': genre_names,
                                'Release Year': release_year
                            })
            else:
                print(f'Erro ao buscar créditos do diretor {director}')
        else:
            print(f'Nenhum resultado encontrado para o diretor {director}')
    else:
        print(f'Erro ao buscar o diretor {director}')
    
    time.sleep(1)

df_all_movies = pd.DataFrame(all_movies)

df_all_movies = df_all_movies.drop_duplicates(subset=['Movie Title'])

df_sortered = df_all_movies.sort_values(by=['Director', 'Release Year'])

KeyboardInterrupt: 